In [1]:
import sys
from pynest import Sampler
import numpy as np
import matplotlib.pyplot as plt
import pymultinest
def simulate_data(a,b,n):
    x = np.linspace(0,10,n)
    y = a*x + b + np.random.normal(loc=0,scale=10,size=n)
    return x,y

x, y = simulate_data(5,10,100)

def log_gaussian(x, mu, sig):
    return np.log(1/np.sqrt(2 * np.pi * sig**2.0))  +  (-0.5/sig**2.0) * (x - mu)**2.0

def myloglike(cube,ndim,nparams):
    log_likelihood = 0
    for i in tuple(zip(x,y)):
        log_likelihood += log_gaussian(i[1], cube[0]*i[0] + cube[1], 10)
    return log_likelihood

def myprior(cube,ndim,nparams):
    cube[0] = cube[0] * 10
    cube[1] = cube[1] * 20
    return cube

def test_mcmc_instantation():
    sampler = Sampler(log_likelihood=myloglike, prior=myprior, ndim = 2, sample_method = 'mcmc')
    sampler.sample()



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



problem: dlopen(/usr/local/lib/libmultinest.dylib, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/9/libgfortran.5.dylib
  Referenced from: /usr/local/lib/libmultinest.3.10.dylib
  Reason: image not found
Traceback (most recent call last):
  File "/Users/rahmanw/opt/anaconda3/envs/bahamas_env/lib/python3.8/site-packages/pymultinest/run.py", line 16, in _load_library
    return cdll.LoadLibrary(libname)
  File "/Users/rahmanw/opt/anaconda3/envs/bahamas_env/lib/python3.8/ctypes/__init__.py", line 451, in LoadLibrary
    return self._dlltype(name)
  File "/Users/rahmanw/opt/anaconda3/envs/bahamas_env/lib/python3.8/ctypes/__init__.py", line 373, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: dlopen(/usr/local/lib/libmultinest.dylib, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/9/libgfortran.5.dylib
  Referenced from: /usr/local/lib/libmultinest.3.10.dylib
  Reason: image not found

During handling of the above exception, another exception occurred:

Traceback (m

SystemExit: 1

/Users/rahmanw/opt/anaconda3/envs/bahamas_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
plt.scatter(x,y,s=5)

In [ ]:
param1 = np.linspace(0,20,100)
param1_likval = [myloglike([5,param]) for param in param1]

In [ ]:
plt.scatter(param1,param1_likval,s=5)
plt.axvline(10)

In [ ]:
param2 = np.linspace(0,10,100)
param2_likval = [myloglike([param,10]) for param in param2]

In [ ]:
plt.scatter(param2,param2_likval,s=5)
plt.axvline(5)

In [ ]:
np.random.normal(loc=0,scale=10)

In [ ]:
sampler = Sampler(log_likelihood=myloglike, prior=myprior, ndim = 2, sample_method = 'mcmc')
sampler.sample()

In [ ]:
pymultinest.run(myloglike, myprior, 2, outputfiles_basename='./2-',resume = False, verbose = True)